#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [1]:
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import datetime
import os

import sqlite3

In [2]:
LOG_BASE_NAME = "log.db"
OPERATION_TYPE = "operation_type"
DURATION = "duration"
EVENT_TIME = "event_time"

with sqlite3.connect(LOG_BASE_NAME) as connection:
    cursor = connection.cursor()
    cursor.execute(
        f'''
        CREATE TABLE IF NOT EXISTS LOG_TABLE (
        {OPERATION_TYPE} TEXT NOT NULL,
        {DURATION} REAL NOT NULL,
        {EVENT_TIME} TEXT
        )
        ''')

In [3]:
# cursor.execute(f'INSERT INTO LOG_TABLE ({OPERATION_TYPE}, {DURATION}, {EVENT_TIME}) VALUES (?, ?, ?)', ('pow', 28, '2023-10-25 18:09:40'))

In [4]:
cursor.execute(f'SELECT * FROM LOG_TABLE')
log_records = cursor.fetchall()
for record in log_records:
    print(record)

('type', 1007.4598789215088, '2023-11-07 16:28:17')
('type', 1006.5948963165283, '2023-11-07 16:28:20')
('type', 1005.305290222168, '2023-11-07 16:28:23')
('type', 992.4256801605225, '2023-11-07 16:28:26')
('type', 1004.1084289550781, '2023-11-07 16:28:29')
('type', 1004.1489601135254, '2023-11-07 16:28:32')
('type', 963.0069732666016, '2023-11-07 16:28:35')
('sum', 1004.2221546173096, '2023-11-07 16:28:38')
('pow', 1005.5267810821533, '2023-11-07 16:28:41')
('ping', 1005.4574012756348, '2023-11-07 17:58:24')
('now', 1006.1662197113037, '2023-11-07 17:58:27')
('type', 1009.1438293457031, '2023-11-07 17:58:30')
('type', 1006.1290264129639, '2023-11-07 17:58:33')
('type', 1010.9927654266357, '2023-11-07 17:58:36')
('type', 1009.4175338745117, '2023-11-07 17:58:39')
('type', 1017.5313949584961, '2023-11-07 17:58:42')
('type', 1028.501272201538, '2023-11-07 17:58:45')
('type', 1004.9893856048584, '2023-11-07 17:58:48')
('sum', 986.6533279418945, '2023-11-07 17:58:51')
('pow', 1003.86977195

In [5]:
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

LOG_NAME = 'log.csv'

server_stats = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler,
                            allow_none=True)


# Добавление строки в лог. max_log_length - максимальное количество записей журнала. 0 - нет ограничения
def add_log(sname, execution_duration_ms, max_log_length):
    if max_log_length > 0:
        with open(LOG_NAME) as f:
            full_log = f.readlines()
        
        # если есть ограничение, и оно превышено
        if len(full_log) >= max_log_length:
            filename = f'{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}log.csv'
            
            # сохранение старого лога и обнуление файла
            with open(filename, 'a') as f_save:
                f_save.writelines(full_log)
            with open(LOG_NAME, 'w'):
                pass
    with open(LOG_NAME, 'a') as f:
        with sqlite3.connect(LOG_BASE_NAME) as log_base_connection:
            cursor = log_base_connection.cursor()
            cursor.execute(f'INSERT INTO LOG_TABLE ({OPERATION_TYPE}, {DURATION}, {EVENT_TIME}) VALUES (?, ?, ?)',
                (str(sname), execution_duration_ms, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
                # f.write(f'{str(sname)}, {execution_duration_ms:2f}ms, {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n')
server_stats.register_function(add_log, 'add_log')

def get_log_records(events: list[str], datetime_start: str, datetime_end: str, duration_min: float, duration_max: float):
    def str_date_to_serverdate(date: str) -> str:
        result = date.replace('-', '').replace(' ', '_').replace(':', '')
        return result

    # log_files_names = [*filter(lambda n: n.endswith(".csv"), os.listdir())]
    # readed_files_names = [*filter(lambda f: (f == LOG_NAME) or (datetime_start <= str_date_to_serverdate(f.split("log")[0]) <= datetime_end), log_files_names)]
    # records = []
    # for f in readed_files_names:
    #     with open(f) as f:
    #         records += [r.split("\n")[0] for r in f.readlines()]
    # result = []
    # for r in records:
    #     operation, duration, date = r.split(', ')
    #     if (datetime_start <= date <= datetime_end):
    #         result += [[operation, duration, date]]

    records_db = []
    with sqlite3.connect(LOG_BASE_NAME) as log_base_connection:
        cursor = log_base_connection.cursor()
        cursor.execute(f'SELECT * FROM LOG_TABLE')
        result = cursor.fetchall()
        cursor.close()
        records_db = [*filter(lambda r: datetime_start <= r[2] <= datetime_end, result)]
        records_db = [*filter(lambda r: duration_min <= r[1] <= duration_max, records_db)]
    # print(records_db)
    return records_db
server_stats.register_function(get_log_records, 'get_log_records')


<function __main__.get_log_records(events: list[str], datetime_start: str, datetime_end: str, duration_min: str, duration_max: str)>

In [6]:
print ("Listening on port 8018...")
server_stats.serve_forever()

Listening on port 8018...


127.0.0.1 - - [07/Nov/2023 19:22:23] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 19:23:26] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2023 19:23:35] "POST /RPC2 HTTP/1.1" 200 -


In [ ]:
get_log_records(["pow", "sum"], "2023-11-07 16:28:28", "3000")

[('type', 1004.1084289550781, '2023-11-07 16:28:29'),
 ('type', 1004.1489601135254, '2023-11-07 16:28:32'),
 ('type', 963.0069732666016, '2023-11-07 16:28:35'),
 ('sum', 1004.2221546173096, '2023-11-07 16:28:38'),
 ('pow', 1005.5267810821533, '2023-11-07 16:28:41')]